In [1]:
import pandas as pd
import re

df = pd.read_csv("../2_flatten_and_clean_addresses/listings_flattened_and_cleaned_address.csv")
df = df.dropna(subset=['BUILDING_NO'])
df['BUILDING_NO'] = df['BUILDING_NO'].astype(str).str.strip()


In [2]:
# 1, 3, 5, 7 - Patterns that represent a unique address as-is
# We copy them directly into CLEAN_BUILDING_NO
pattern_unique = (
    r'^\d+$|'                    # 1 - Plain number (e.g. 2216)
    r'^\d+[A-Z]$|'               # 3 - Number with letter suffix (e.g. 1684A)
    r'^\d+-\d+$|'                # 5 - Queens hyphenated number (e.g. 63-18)
    r'^\d+-\d+[A-Z]$'            # 7 - Hyphenated with letter suffix (e.g. 34-72A)
)

df_unique = df[df['BUILDING_NO'].str.match(pattern_unique)]
df_unique = df_unique.copy()
df_unique['CLEAN_BUILDING_NO'] = df_unique['BUILDING_NO']
df_unique.head()


,BOROUGH,ZIP,BUILDING_NO,STREET,BLOCK,LOT,COUNTY,CITY,STATUS1,STATUS2,STATUS3,CLEAN_BUILDING_NO
0,MANHATTAN,10001,246,10th Avenue,722.0,3.0,62,NEW YORK,MULTIPLE DWELLING A,NaN,NaN,246
1,MANHATTAN,10001,299,10th Avenue,699.0,31.0,62,NEW YORK,MULTIPLE DWELLING A,NaN,NaN,299
2,MANHATTAN,10001,301,10th Avenue,699.0,32.0,62,NEW YORK,MULTIPLE DWELLING A,NaN,NaN,301
4,MANHATTAN,10001,440,10th Avenue,732.0,73.0,62,NEW YORK,MULTIPLE DWELLING A,NaN,NaN,440
5,MANHATTAN,10001,442,10th Avenue,732.0,72.0,62,NEW YORK,MULTIPLE DWELLING A,NaN,NaN,442


In [3]:
# 2 - Plain Range
# Example: '953 TO 957' → ['953', '954', '955', '956', '957']

df_plain_range = df[df['BUILDING_NO'].str.match(r'^\d+ TO \d+$')].copy()

def expand_plain_range_row(row):
    start, end = map(int, row['BUILDING_NO'].split(' TO '))
    return pd.DataFrame([
        {**row, 'CLEAN_BUILDING_NO': str(n)} for n in range(start, end + 1)
    ])

df_expanded_plain = pd.concat(
    [expand_plain_range_row(row) for _, row in df_plain_range.iterrows()],
    ignore_index=True
)

df_expanded_plain[df_expanded_plain['CLEAN_BUILDING_NO'].isnull()]

,BOROUGH,ZIP,BUILDING_NO,STREET,BLOCK,LOT,COUNTY,CITY,STATUS1,STATUS2,STATUS3,CLEAN_BUILDING_NO


In [4]:
# 4 - Fraction / decimal
# Example: '108.5' → '108 1/2'

df_fracdec = df[df['BUILDING_NO'].str.match(r'^\d+\.5$|^\d+ \d+/\d+$', na=False)].copy()

df_fracdec['CLEAN_BUILDING_NO'] = df_fracdec['BUILDING_NO'].str.replace(r'\.5$', ' 1/2', regex=True)
df_fracdec.head()

df_fracdec[~df_fracdec['CLEAN_BUILDING_NO'].str.endswith(' 1/2', na=False)]

,BOROUGH,ZIP,BUILDING_NO,STREET,BLOCK,LOT,COUNTY,CITY,STATUS1,STATUS2,STATUS3,CLEAN_BUILDING_NO


In [5]:
# 6 - Hyphenated Range
# Example: '122-05 TO 122-07' → ['122-05', '122-06', '122-07']

df_hyphen_range = df[df['BUILDING_NO'].str.match(r'^\s*\d{1,3}-\d{1,3}\s*TO\s*\d{1,5}(-\d{1,3})?\s*$'
)].copy()

def expand_hyphen_range_row(row):
    start, end = row['BUILDING_NO'].split(' TO ')
    if '-' not in end:
        row['CLEAN_BUILDING_NO'] = start.strip()
        return pd.DataFrame([row])

    prefix_start, suffix_start = start.strip().split('-')
    prefix_end, suffix_end = end.strip().split('-')

    prefix_start_n = int(prefix_start)
    prefix_end_n = int(prefix_end)
    suffix_width = len(suffix_start)

    rows = []
    for p in range(prefix_start_n, prefix_end_n + 1):
        suffix_start_n = int(suffix_start) if p == prefix_start_n else 1
        suffix_end_n = int(suffix_end) if p == prefix_end_n else 99
        for s in range(suffix_start_n, suffix_end_n + 1):
            rows.append({**row, 'CLEAN_BUILDING_NO': f"{p}-{str(s).zfill(suffix_width)}"})

    return pd.DataFrame(rows)

df_expanded_hyphen = pd.concat(
    [expand_hyphen_range_row(row) for _, row in df_hyphen_range.iterrows()],
    ignore_index=True
)
df_expanded_hyphen.head()

df_expanded_hyphen[df_expanded_hyphen['CLEAN_BUILDING_NO'].isnull()]


,BOROUGH,ZIP,BUILDING_NO,STREET,BLOCK,LOT,COUNTY,CITY,STATUS1,STATUS2,STATUS3,CLEAN_BUILDING_NO


In [6]:
# 8 - Range with letter suffix
# Handles: '222R TO 224R' → ['222R', '223R', '224R']
#      AND '711A TO 711D' → ['711A', '711B', '711C', '711D']

# Pattern for varying number, same letter
pattern_num_vary = r'^(\d+)([A-Z]) TO (\d+)\2$'
# Pattern for same number, varying letter
pattern_letter_vary = r'^(\d+)([A-Z]) TO \1([A-Z])$'

# Filter rows matching either pattern
mask = (
    df['BUILDING_NO'].str.contains(r' TO ', na=False) &
    (
        df['BUILDING_NO'].str.match(pattern_num_vary, na=False) |
        df['BUILDING_NO'].str.match(pattern_letter_vary, na=False)
    )
)
df_ranges_to_expand = df[mask].copy()

def expand_letter_range_row(row):
    building_no = row['BUILDING_NO']
    # Keep all original data, remove only CLEAN_BUILDING_NO if it exists
    original_row_data = row.to_dict()
    original_row_data.pop('CLEAN_BUILDING_NO', None) # Remove placeholder if present

    match_num = re.match(pattern_num_vary, building_no)
    if match_num: # Case 1: Varying Number, Same Letter
        start_num_str, letter, end_num_str = match_num.groups()
        start_num, end_num = int(start_num_str), int(end_num_str)
        if start_num > end_num: start_num, end_num = end_num, start_num
        return pd.DataFrame([
            # Keep original BUILDING_NO, add CLEAN_BUILDING_NO
            {**original_row_data, 'CLEAN_BUILDING_NO': str(n) + letter}
            for n in range(start_num, end_num + 1)
        ])

    match_letter = re.match(pattern_letter_vary, building_no)
    if match_letter: # Case 2: Same Number, Varying Letter
        num_part, start_letter, end_letter = match_letter.groups()
        start_ord, end_ord = ord(start_letter), ord(end_letter)
        if start_ord > end_ord: start_ord, end_ord = end_ord, start_ord
        return pd.DataFrame([
            # Keep original BUILDING_NO, add CLEAN_BUILDING_NO
            {**original_row_data, 'CLEAN_BUILDING_NO': num_part + chr(c)}
            for c in range(start_ord, end_ord + 1)
        ])

    # Fallback: return DataFrame based on original row's columns + CLEAN_BUILDING_NO
    # Ensure BUILDING_NO is included in the columns list
    cols = list(original_row_data.keys()) + ['CLEAN_BUILDING_NO']
    return pd.DataFrame(columns=cols)


# Apply the unified function
df_expanded_letter_range = pd.concat(
    [expand_letter_range_row(row) for _, row in df_ranges_to_expand.iterrows()],
    ignore_index=True
)
# Ensure no completely empty rows slipped through if expansion failed
df_expanded_letter_range = df_expanded_letter_range.dropna(subset=['CLEAN_BUILDING_NO'])

df_expanded_letter_range[df_expanded_letter_range['CLEAN_BUILDING_NO'].isnull()]

,BOROUGH,ZIP,BUILDING_NO,STREET,BLOCK,LOT,COUNTY,CITY,STATUS1,STATUS2,STATUS3,CLEAN_BUILDING_NO


In [7]:
# 9 - Hyphenated range with letter suffix
# Example: '143-19A TO 143-25A' → ['143-19A', '143-20A', ..., '143-25A']

df_hyphen_letter_range = df[df['BUILDING_NO'].str.match(r'^\d+-\d+[A-Z] TO \d+-\d+[A-Z]$', na=False)].copy()

def expand_hyphen_letter_range_row(row):
    def split_parts(val):
        match = re.match(r'^(\d+)-(\d+)([A-Z])$', val)
        return match.group(1), int(match.group(2)), match.group(3)

    start, end = row['BUILDING_NO'].split(' TO ')
    prefix_start, num_start, letter_start = split_parts(start)
    prefix_end, num_end, letter_end = split_parts(end)

    if prefix_start != prefix_end:
        row['CLEAN_BUILDING_NO'] = None
        return pd.DataFrame([row])

    result = []
    width = len(re.match(r'^(\d+)-(\d+)([A-Z])$', start).group(2))

    for i in range(num_start, num_end + 1):
        letter_from = letter_start if i == num_start else 'A'
        letter_to = letter_end if i == num_end else 'B'

        for code in range(ord(letter_from), ord(letter_to) + 1):
            result.append({**row, 'CLEAN_BUILDING_NO': f"{prefix_start}-{str(i).zfill(width)}{chr(code)}"})

    return pd.DataFrame(result)

# Run the expansion
df_expanded_hyphen_letter = pd.concat(
    [expand_hyphen_letter_range_row(row) for _, row in df_hyphen_letter_range.iterrows()],
    ignore_index=True
)

df_expanded_hyphen_letter[df_expanded_hyphen_letter['CLEAN_BUILDING_NO'].isnull()]



,BOROUGH,ZIP,BUILDING_NO,STREET,BLOCK,LOT,COUNTY,CITY,STATUS1,STATUS2,STATUS3,CLEAN_BUILDING_NO


In [11]:
# Concatenate all sub-dataframes into one
all_dfs = [
    df_unique,
    df_expanded_plain,
    df_fracdec,
    df_expanded_hyphen,
    df_expanded_letter_range,
    df_expanded_hyphen_letter
]
final_df = pd.concat(all_dfs, ignore_index=True)
final_df.head()

final_df.to_csv('listings_with_clean_building_no.csv', index=False)